# 18-4-3-DRILL-Interpreting-coefficients

## Interpreting-coefficients

To close out this checkpoint, you're going to do three assignments. For the first assignment, you'll write up a short answer to a question. For the second two assignments, you'll do your work in Jupyter notebooks.

Please submit links to all your work below. This is not a graded checkpoint, but you should discuss your solutions with your mentor. Also, when you're done, compare your work to these example solutions. https://github.com/Thinkful-Ed/machine-learning-regression-problems/blob/master/notebooks/4.solution_understanding_the_relationship.ipynb

3. House prices model
In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

    * Load the houseprices data from Thinkful's database.

    * Run your house prices model again and interpret the results. Which features are 
    statistically significant, and which are not?

    * Now, exclude the insignificant features from your model. Did anything change?

    * Interpret the statistically significant coefficients by quantifying their relations with 
    the house prices. Which features have a more prominent effect on house prices?

* Do the results sound reasonable to you? If not, try to explain the potential reasons.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
#postgres_user = 'dsbc_student'
#postgres_pw = '7*.8G9QH21'
#postgres_host = '142.93.121.174'
#postgres_port = '5432'
#postgres_db = 'weatherinszeged'

### Load the data 

In [2]:
#engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
#    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
#houseprices = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
#engine.dispose()



houseprices = pd.read_csv('housing-train.csv')

houseprices.head(10)

#weather.head(10)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.000,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.000,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,nan,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.000,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.000,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [3]:
#houseprices.info()

In [7]:
# extracting just the variables needed 
housePrices_df = houseprices.iloc[:,[18,19,49,50,51,56, 61,80]]

In [ ]:
## Plot the scatter matrix for all variables in the dataset. 
##Do all variables have a pairwise linear relationship?

#pd.plotting.scatter_matrix(weather, figsize=(12, 12))
#plt.tight_layout()
#plt.show()

In [8]:
# Next check the data to make sure there is no missing data. 
# Remove rows containing missing data if it exists. 

housePrices_df.isnull().sum()*100/housePrices_df.isnull().count()

OverallCond    0.000
YearBuilt      0.000
FullBath       0.000
HalfBath       0.000
BedroomAbvGr   0.000
Fireplaces     0.000
GarageCars     0.000
SalePrice      0.000
dtype: float64

In [9]:
### rename columns
housePrices_df.rename(columns={'OverallCond': 'overallcond', 
                                'YearBuilt': 'yearbuilt', 'FullBath': 'fullbath', 
                                'HalfBath': 'halfbath', 'BedroomAbvGr': 'bedroomabvgr', 'Fireplaces': 'fireplaces',
                                'GarageCars': 'garagecars', 'SaleCondition': 'salecondition', 'SalePrice': 'saleprice'}, inplace=True)

In [10]:
# Y is the target variable
Y = housePrices_df['saleprice'].values

# X is the feature set which includes  
X = housePrices_df[['overallcond', 'yearbuilt', 'fullbath', 'halfbath', 'bedroomabvgr','fireplaces','garagecars']]

In [11]:
### Double check that the model is correct
import statsmodels.api as sm

# We need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     314.2
Date:                Wed, 08 Jan 2020   Prob (F-statistic):          2.10e-285
Time:                        23:02:21   Log-Likelihood:                -17871.
No. Observations:                1460   AIC:                         3.576e+04
Df Residuals:                    1452   BIC:                         3.580e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.163e+06    1.2e+05     -9.692      0.000    -1.4e+06   -9.28e+05
overallcond   8931.2203   1277.149      6.993      0.000    6425.967    1.14e+04
yearbuilt      581.8616     60.508      9.616      0.000     463.170     700.554
fullbath      3.514e+04   3178.272     11.056      0.000    2.89e+04    4.14e+04
halfbath       1.17e+04   2848.351      4.106      0.000    6109.119    1.73e+04
bedroomabvgr  1965.0399   1878.323      1.046      0.296   -1719.476    5649.556
fireplaces    3.206e+04   2184.994     14.672      0.000    2.78e+04    3.63e+04
garagecars    3.551e+04   2243.336     15.827      0.000    3.11e+04    3.99e+04
==============================================================================
Omnibus:                      712.576   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7410.220
Skew:                           2.021   Prob(JB):                         0.00
Kurtosis:                      13.270   Cond. No.                     1.80e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.8e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* Other than "bedroomabvgr" whose pvalue > 0.05 which means that that feature is not significant - all other features are significant in determining sale price.

* Now, exclude the insignificant features from your model. Did anything change?


In [12]:
# Y is the target variable
Y = housePrices_df['saleprice'].values

# X is the feature set which includes  
X = housePrices_df[['overallcond', 'yearbuilt', 'fullbath', 'halfbath','fireplaces','garagecars']]

In [13]:
### Double check that the model is correct
import statsmodels.api as sm

# We need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     366.4
Date:                Wed, 08 Jan 2020   Prob (F-statistic):          1.81e-286
Time:                        23:05:37   Log-Likelihood:                -17871.
No. Observations:                1460   AIC:                         3.576e+04
Df Residuals:                    1453   BIC:                         3.579e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.122e+06   1.13e+05     -9.903      0.000   -1.34e+06      -9e+05
overallcond  8911.7170   1277.054      6.978      0.000    6406.650    1.14e+04
yearbuilt     562.3854     57.575      9.768      0.000     449.447     675.324
fullbath     3.661e+04   2851.908     12.836      0.000     3.1e+04    4.22e+04
halfbath     1.249e+04   2744.856      4.551      0.000    7108.434    1.79e+04
fireplaces   3.203e+04   2184.935     14.661      0.000    2.77e+04    3.63e+04
garagecars   3.549e+04   2243.353     15.820      0.000    3.11e+04    3.99e+04
==============================================================================
Omnibus:                      706.165   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7273.033
Skew:                           2.000   Prob(JB):                         0.00
Kurtosis:                      13.176   Cond. No.                     1.70e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.7e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

 * Interpret the statistically significant coefficients by quantifying their relations with 
    the house prices. 

 - const	-1.122e+06
- overallcond	8911.7170
- yearbuilt	562.3854
- fullbath	3.661e+04
- halfbath	1.249e+04
- fireplaces	3.203e+04
- garagecars	3.549e+04

According to the estimation results:
1 point increase in overallqual results in $8911 increase in sale price.
1 point increase in yearbuilt results in $562 increase in sale price.
1 point increase in fullbath results in $36610 increase in sale price.
1 point increase in halfbath results in $12490 increase in sale price.
1 point increase in fireplaces results in $32030 increase in sale price.
1 point increase in garagecars results in $35490 increase in sale price.

## DO I HAVE TO USE Categorical valiable here? like neighborhood or zoning? - really dont' want to use dummy variables - what are the pros and cons for using the dummy variables?

##  Which features have a more prominent effect on house prices?
Considering the results, the most prominent factors affecting the sale price seem to be the fullbath, fireplaces, and garagecars, and then halfbath.

## * Do the results sound reasonable to you? If not, try to explain the potential reasons.

- There are many features - I only picked a few. A better way to do this experiement would be to look at all the features and choose the ones that are most significant. This will be a long list and will tak ea lot of run time - but might give us a better estimate. 

Otherwise it seems like reasonable - all variables are continuous so there are no dummy variables.